In [144]:
import pandas as pd
import numpy as np
from scipy import sparse
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from sklearn.metrics.pairwise import cosine_similarity

In [145]:
def rating_movie_df():
    ratings = pd.read_csv('../data/ml-latest-small/ratings.csv')
    movies = pd.read_csv('../data/ml-latest-small/movies.csv')
    return ratings,movies

In [146]:
def user_movie_mat(df,user_col, item_col, rating_col):
    umd = df.groupby([user_col, item_col])[rating_col].sum().unstack().reset_index().fillna(0).set_index(user_col)
    return umd

In [147]:
def user_to_id(usr_movie_mat):
    user_id = list(usr_movie_mat.index)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict

In [148]:
def movie_to_id(df,id_col,name_col):
    item_dict ={}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i,name_col]
    return item_dict

In [149]:
def get_FM_model(user_item_mat, n_components=30, loss='warp', k=15, epoch=30, n_jobs = 4):
    x = sparse.csr_matrix(user_item_mat.values)
    model = LightFM(no_components= n_components, loss=loss,k=k)
    model.fit(x,epochs=epoch,num_threads = n_jobs)
    
    train_precision = precision_at_k(model, x, k=10).mean()
    test_precision = precision_at_k(model, x, k=10).mean()
    train_auc = auc_score(model, x).mean()
    test_auc = auc_score(model, x).mean()
    print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
    print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))
    
    return model

In [150]:
def sample_recommendation_user(model, user_item_mat, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 10):
    n_users, n_items = user_item_mat.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    scores.index = user_item_mat.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(user_item_mat.loc[user_id,:] \
                                 [user_item_mat.loc[user_id,:] > threshold].index) \
								 .sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))

    print("Known Likes:")
    counter = 1
    for i in known_items:
        print(str(counter) + '- ' + i)
        counter+=1

    print("\n Recommended Items:")
    counter = 1
    for i in scores:
        print(str(counter) + '- ' + i)
        counter+=1
    return return_score_list

In [151]:
def sample_recommendation_item(model,interactions,item_id,user_dict,item_dict,number_of_user):
    n_users, n_items = interactions.shape
    x = np.array(interactions.columns)
    scores = pd.Series(model.predict(np.arange(n_users), np.repeat(x.searchsorted(item_id),n_users)))
    user_list = list(interactions.index[scores.sort_values(ascending=False).head(number_of_user).index])
    return user_list 



In [152]:
def create_item_emdedding_distance_matrix(model,interactions):
    df_item_norm_sparse = sparse.csr_matrix(model.item_embeddings)
    similarities = cosine_similarity(df_item_norm_sparse)
    item_emdedding_distance_matrix = pd.DataFrame(similarities)
    item_emdedding_distance_matrix.columns = interactions.columns
    item_emdedding_distance_matrix.index = interactions.columns
    return item_emdedding_distance_matrix

In [153]:
def item_item_recommendation(item_emdedding_distance_matrix, item_id, 
                             item_dict, n_items = 10, show = True):
    recommended_items = list(pd.Series(item_emdedding_distance_matrix.loc[item_id,:]. \
                                  sort_values(ascending = False).head(n_items+1). \
                                  index[1:n_items+1]))
    if show == True:
        print("Item of interest :{0}".format(item_dict[item_id]))
        print("Item similar to the above item:")
        counter = 1
        for i in recommended_items:
            print(str(counter) + '- ' +  item_dict[i])
            counter+=1
    return recommended_items

In [154]:
if __name__=='__main__':
    ratings, movie = rating_movie_df()
    umd = user_movie_mat(df = ratings,user_col = 'userId', item_col = 'movieId', rating_col = 'rating')
    user_dict = user_to_id(umd)
    movies_dict = movie_to_id(df = movie,id_col = 'movieId',name_col = 'title')
    mf_model = get_FM_model(umd,n_components = 30,loss = 'warp',epoch = 30,n_jobs = 4)

Precision: train 0.65, test 0.65.
AUC: train 0.99, test 0.99.


In [155]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [156]:
 movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [157]:
interactions.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [158]:
rec_list = item_item_recommendation(item_emdedding_distance_matrix = item_item_dist,
                                    item_id = 5378,
                                    item_dict = movies_dict,
                                    n_items = 10)
print("Recommendation list is")
print(rec_list)

Item of interest :Star Wars: Episode II - Attack of the Clones (2002)
Item similar to the above item:
1- Spider-Man (2002)
2- Lord of the Rings: The Fellowship of the Ring, The (2001)
3- Minority Report (2002)
4- Star Wars: Episode III - Revenge of the Sith (2005)
5- Lord of the Rings: The Two Towers, The (2002)
6- Matrix Reloaded, The (2003)
7- Matrix Revolutions, The (2003)
8- Unbreakable (2000)
9- Bourne Supremacy, The (2004)
10- X-Men (2000)
Recommendation list is
[5349, 4993, 5445, 33493, 5952, 6365, 6934, 3994, 8665, 3793]


In [160]:
rec_list = sample_recommendation_user(model = mf_model, 
                                  interactions = interactions, 
                                  user_id = 11, 
                                  user_dict = user_dict,
                                  item_dict = movies_dict, 
                                  threshold = 4,
                                  nrec_items = 10)

TypeError: sample_recommendation_user() got an unexpected keyword argument 'interactions'

In [ ]:
item_item_dist = create_item_emdedding_distance_matrix(model = mf_model,
                                                       interactions = interactions)

print()

rec_list = item_item_recommendation(item_emdedding_distance_matrix = item_item_dist,
                                item_id = 5378,
                                item_dict = movies_dict,
                                n_items = 10)
print("Recommendation list is")
print(rec_list)

In [ ]:
rec_list = item_item_recommendation(item_emdedding_distance_matrix = item_item_dist,
                                    item_id = 5378,
                                    item_dict = movies_dict,
                                    n_items = 10)